In [ ]:
class NeurNet(tf.Module):
    def __init__(self, sigma_w, sigma_b, Nboot, Mu_, STD_, activation, depth, input_size, hidden_size, output_size):
        super().__init__()
        #self.optimizer = tf.keras.optimizers.Adam()
        self.activation = activation
        self.lsizes = np.concatenate(([input_size], np.repeat(hidden_size, depth-1), [output_size]))
        self.sigma_w = sigma_w
        self.sigma_b = sigma_b
        self.depth = depth
        self.mu = Mu_
        self.std_dev = STD_
        self.Nboot = Nboot
        #self.Ws = np.empty((Nboot, depth), dtype=object)  
        #self.Bs = np.empty((Nboot, depth), dtype=object)
        self.Ws = self.initialize_weights()
        self.Bs = self.initialize_biases()
        '''
        for i in range(Nboot):
            for l in range(depth):
                bname = 'b{}'.format(l)
                Wname = 'W{}'.format(l)
                self.Ws[i, l] = tf.Variable(tf.random.normal([self.lsizes[l], self.lsizes[l+1]], mean=Mu_, stddev=STD_), name=Wname)
                self.Bs[i, l] = tf.Variable(tf.random.normal([self.lsizes[l+1]], mean=Mu_, stddev=STD_), name=bname)
        '''
    def initialize_weights(self):
        Ws = np.empty((self.Nboot, self.depth), dtype=object)
        for i in range(self.Nboot):
            for l in range(self.depth):
                Wname = 'W{}'.format(l)
                Ws[i, l] = tf.Variable(tf.random.normal([self.lsizes[l], self.lsizes[l+1]], mean=self.mu, stddev=self.std_dev), name=Wname)
        return Ws

    def initialize_biases(self):
        Bs = np.empty((self.Nboot, self.depth), dtype=object)
        for i in range(self.Nboot):
            for l in range(self.depth):
                bname = 'b{}'.format(l)
                Bs[i, l] = tf.Variable(tf.random.normal([self.lsizes[l+1]], mean=self.mu, stddev=self.std_dev), name=bname)
        return Bs
    
    def forward(self, X, i):
        with tf.GradientTape() as tape:
            tape.watch([param for param in [self.Ws[i, j] for j in range(self.depth)] + [self.Bs[i, j] for j in range(self.depth)]])

            A = X
            for j in range(self.depth):
                Z = tf.add(tf.matmul(A, self.sigma_w/self.lsizes[j]**0.5*self.Ws[i, j]), self.sigma_b*self.Bs[i, j])
                A = self.activation(Z)

            Z_last = Z

            #gradients = tape.gradient(Z_last, [self.Ws[i, j] for j in range(self.depth)] + [self.Bs[i, j] for j in range(self.depth)])
        return Z_last#, gradients
    
    @tf.function
    def train_step(self, X_train, Y_train, eta, i):
        with tf.GradientTape() as tape:
            Z = self.forward(X_train, i)
            loss = tf.losses.mean_squared_error(Z, Y_train)  # Define loss func using Z and x_train
            optimizer = tf.keras.optimizers.Adam()
            #variables = [param for param in [self.Ws[i, j] for j in range(self.depth)] + [self.Bs[i, j] for j in range(self.depth)]]
            variables = [param for param in [self.Ws[i, j] for j in range(self.depth)] + [self.Bs[i, j] for j in range(self.depth)]]
            optimizer.build(variables)
            gradients = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(gradients, variables))

        return Z, loss
    
    def training_measure(self, X_train, Y_train, eta, num_epochs, X_out, Nout, i):
        self.num_epochs = num_epochs
        self.obser_sum = np.ndarray(shape=(num_epochs, 3, Nout, self.Nboot), dtype=float, buffer=np.zeros((num_epochs, 3, Nout, self.Nboot)))
        for epoch in range(num_epochs):
            Z, loss = self.train_step(X_train, Y_train, eta, i)
            loss_value = tf.reduce_mean(loss)

            Z = self.forward(X_out, i)
            Z2 = tf.square(Z)
            #print(Z.numpy())
            self.obser_sum[epoch, 0, :, i] = np.reshape(Z.numpy(), (Nout,) )
            self.obser_sum[epoch, 1, :, i] = np.reshape(Z2.numpy(), (Nout,) )
            #obser_sum[epoch, 2, :, i] = np.reshape(loss.numpy(), (Nout,) )

In [ ]:
Nin = 2
Nout = 100
xx1 = tf.constant(-1., shape=(Nin, 1, 1), dtype=tf.float32)
xx2 = tf.constant(1., shape=(Nin, 1, 1), dtype=tf.float32)

def interp(_alpha):
    condition1 = tf.logical_and(_alpha <= 1, _alpha >= 0)
    condition2 = _alpha > 1
    condition3 = _alpha < 0
    
    result = tf.where(condition1, _alpha * xx1 + (1 - _alpha) * xx2, tf.where(condition2, xx1, xx2))
    return result

alpha_train = tf.random.uniform(shape=(Nin,1,1), minval=-2, maxval=3, dtype=tf.float32)
x_train = interp(alpha_train)

alpha_out = tf.random.uniform(shape=(Nout,1,1), minval=-2, maxval=3, dtype=tf.float32)

#print(x_train)
#sys.exit()

sigma_w = np.sqrt(1.5)
sigma_b = 0.
Mu = 0.
STD = 1.
Nboot = 10
depth = 3
input_size = 1
hidden_size = 128
output_size = 1
eta = 0.5

# Numb of train epochs
num_epochs = 500

rete = NeurNet(sigma_w, sigma_b, Nboot, Mu, STD, tf.nn.tanh, depth, input_size, hidden_size, output_size)

for i in range(Nboot):
    rete.training_measure(alpha_train, x_train, eta, num_epochs, alpha_out, Nout, i)
'''
lsizes = np.concatenate(([input_size], np.repeat(hidden_size, depth-1), [output_size]))
#print(lsizes)

#loss_fun = tf.losses.mean_squared_error()

# obser_sum = [mean output net, dev_std output net, loss]
obser_sum = np.ndarray(shape=(num_epochs, 3, Nout, Nboot), dtype=float, buffer=np.zeros((num_epochs, 3, Nout, Nboot)))

for i in range(Nboot):

    for l in range(depth):
        Ws[i, l], Bs[i, l] = initialization(lsizes[l], lsizes[l+1], Mu, STD / lsizes[l]**0.5, l)
        #print(l, Ws[i,l].shape)

    # Optimizer definition 
    optimizer = tf.keras.optimizers.Adam()

    # Train function
    @tf.function
    def train_step(X, Y):
        with tf.GradientTape() as tape:
            Z, gradients = forward(X, i, tf.nn.tanh, depth)
            loss = tf.losses.mean_squared_error(Z, x_train)  # Define loss func using Z and x_train

        variables = [param for param in [Ws[i, j] for j in range(depth)] + [Bs[i, j] for j in range(depth)]]
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

        return Z, loss

    # Train cycle
    for epoch in range(num_epochs):
        Z, loss = train_step(alpha_train, x_train)
        loss_value = tf.reduce_mean(loss)
        
        Z, _ = forward(alpha_out, i, tf.nn.tanh, depth)
        Z2 = tf.square(Z)
        #print(Z.numpy())
        obser_sum[epoch, 0, :, i] = np.reshape(Z.numpy(), (Nout,) )
        obser_sum[epoch, 1, :, i] = np.reshape(Z2.numpy(), (Nout,) )
        #obser_sum[epoch, 2, :, i] = np.reshape(loss.numpy(), (Nout,) )
'''
alpha_np = alpha_train.numpy()

In [ ]:
xx1 = tf.constant(-1., shape=(Nout, 1, 1), dtype=tf.float32)
xx2 = tf.constant(1., shape=(Nout, 1, 1), dtype=tf.float32)

alpha_np = alpha_out.numpy()
alpha_np = np.reshape(alpha_out, (Nout,))

x_np = interp(alpha_out)
x_np = np.reshape(x_np.numpy(), (Nout,))

epoch_fix = 499

y = np.array([0.]*len(alpha_np))
erry = np.array([0.]*len(alpha_np))

for i in range(len(alpha_np)):
    y[i] = mean(rete.obser_sum[epoch_fix, 0, i, :])
    erry[i] = np.sqrt(mean(rete.obser_sum[epoch_fix, 1, i, :]) - y[i]**2.)#std_dev(Nboot*obser_sum[epoch_fix, 0, i, :])

print(erry)

#print(mean(obser_sum[epoch_fix, 0, 0, :]), std_dev(obser_sum[epoch_fix, 0, 0, :])*Nboot**0.5, (mean(obser_sum[epoch_fix, 1, 0, :]) - mean(obser_sum[epoch_fix, 0, 0, :])**2.)**0.5)

#err = np.sqrt(obser_sum[epoch_fix, 1, :] - obser_sum[epoch_fix, 0, :]**2.)
#print(alpha_np[0:5], x_np[0:5], obser_sum[400, 0, 0:5], obser_sum[400, 2, 0:5])
plt.errorbar(alpha_np, y, yerr=2.*erry, fmt = '.', capsize=1)
plt.errorbar(alpha_np, x_np, fmt = '.', capsize=1)
plt.errorbar(np.reshape(alpha_train.numpy(), (Nin, )), np.reshape(x_train.numpy(), (Nin,)), fmt = 'o', capsize=30)
plt.show()

# PUT THE BIAS AT ZERO AND CHANGE THE DEV_STD